In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
import random
import math
import re
import time
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg

from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn.visualize import display_instances
import mrcnn.model as modellib
from mrcnn.model import log
from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Root directory of the project
ROOT_DIR = "C:/Users/dymon/OneDrive/Desktop/rep2"

DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

MODEL_DIR = os.path.join(ROOT_DIR, "logs")


WEIGHTS_PATH = "C:/Users/dymon/OneDrive/Desktop/rep2/mask_rcnn_object_0020.h5"

Using TensorFlow backend.


In [2]:
class CustomConfig(Config):
    """Configuration for training on the custom  dataset.
    Derives from the base Config class and overrides some values.
    """
    NAME = "object"

    IMAGES_PER_GPU = 1

    NUM_CLASSES = 1 + 2  # Background + benign tumors + malignant tumors

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 51

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

In [3]:
# Code for Customdataset class. Same code is present in custom.py file also
class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):

        self.add_class("object", 1, "benign")
        self.add_class("object", 2, "malignant")

        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)
        annotations1 = json.load(open('C:/Users/dymon/OneDrive/Desktop/rep2/dataset/train/train_json.json'))

        annotations = list(annotations1.values())  # don't need the dict keys

        annotations = [a for a in annotations if a['regions']]
        
        # Add images
        for a in annotations:
           
            polygons = [r['shape_attributes'] for r in a['regions']] 
            objects = [s['region_attributes']['names'] for s in a['regions']]
            print("objects:",objects)
            name_dict = {"benign": 1,"malignant": 2}
            num_ids = [name_dict[a] for a in objects]

            print("numids",num_ids)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object", 
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):
       
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):

            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

            mask[rr, cc, i] = 1

        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [4]:
def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = CustomDataset()
    dataset_train.load_custom("C:/Users/dymon/OneDrive/Desktop/rep2/dataset", "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomDataset()
    dataset_val.load_custom("C:/Users/dymon/OneDrive/Desktop/rep2/dataset", "val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=10,
                layers='heads')

                            
config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)

weights_path = WEIGHTS_PATH
       
if not os.path.exists(weights_path):
  utils.download_trained_weights(weights_path)

model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

train(model)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
objects: ['benign']
numids [1]
objects: ['benign']
numids [1]
objects: ['malignant']
numids [2]
objects: ['benign', 'benign']
numids [1, 1]
objects: ['benign']
numids [1]
objects: ['benign']
numids [1]
objects: ['benign']
numids [1]
objects: ['benign']
numids [1]
objects: ['benign']
numids [1]
objects: ['benign']
numids [1]
objects: ['benign']
numids [1]
objects: ['benign']
numids [1]
objects: ['benign']
numids [1]
objects: ['benign']
numids [1]
objects: ['benign']
numids [1]
objects: ['benign']
numids [1]
objects: ['benign']
numids [1]
objects: ['benign']
numids [1]
objects: ['benign']
numids [1]
objects: ['benign']
numids [1]
objects: ['benign']
numids [1]
objects: ['benign']
numids [1]
objects: ['benign']
numids [1]
objects: ['benign']
numids [1]
objects: ['benign', '

Epoch 1/10
51/51 [==============================] - 4538s - loss: 1.9282 - rpn_class_loss: 0.0050 - rpn_bbox_loss: 0.1073 - mrcnn_class_loss: 0.1505 - mrcnn_bbox_loss: 0.5296 - mrcnn_mask_loss: 1.1357 - val_loss: 12.8943 - val_rpn_class_loss: 0.1555 - val_rpn_bbox_loss: 12.7387 - val_mrcnn_class_loss: 2.3842e-07 - val_mrcnn_bbox_loss: 0.0000e+00 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 2/10
51/51 [==============================] - 3514s - loss: 0.8764 - rpn_class_loss: 0.0044 - rpn_bbox_loss: 0.1353 - mrcnn_class_loss: 0.0633 - mrcnn_bbox_loss: 0.3591 - mrcnn_mask_loss: 0.3143 - val_loss: 14.2615 - val_rpn_class_loss: 0.1611 - val_rpn_bbox_loss: 14.1004 - val_mrcnn_class_loss: 1.1921e-07 - val_mrcnn_bbox_loss: 0.0000e+00 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 3/10
51/51 [==============================] - 4021s - loss: 0.6553 - rpn_class_loss: 0.0028 - rpn_bbox_loss: 0.0929 - mrcnn_class_loss: 0.0590 - mrcnn_bbox_loss: 0.2278 - mrcnn_mask_loss: 0.2728 - val_loss: 16.1717 - val_rpn_class